In [1]:

import dagshub
dagshub.init(repo_owner='Vaibha3246', repo_name='influence_mirror', mlflow=True)

import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/Vaibha3246/influence_mirror.mlflow")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

e:\influence-mirror\myenv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=576d6d2d-de84-4a13-b6bd-d56f6656e8d6&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=3781d83efd1b725b3a8791420d6063319b2b089c4bfc1c05ab609d2c8069b0dc




JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
# Set or create an experiment
mlflow.set_experiment("TfIdf trigram max_features")

2025/09/29 15:58:16 INFO mlflow.tracking.fluent: Experiment with name 'TfIdf trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/c4fa29dc2c024ffb952e922d308a399f', creation_time=1759141695620, experiment_id='2', last_update_time=1759141695620, lifecycle_stage='active', name='TfIdf trigram max_features', tags={}>

In [3]:
import pandas as pd
df=pd.read_csv('preprocessing.csv')

In [4]:
df.head(5)

,video_id,category,text,likes,published_at,text_clean,sentiment,sentiment_numeric,word_count,num_stop_words,...,fear,anger,anticipation,trust,surprise,positive,negative,sadness,disgust,joy
0,4wZwXhoxRIA,technology,All products can be found on www.justicebuys.c...,1978,2025-01-04 19:28:08+00:00,product found 🙌🏼 since review 50+ product per ...,positive,1,24,9,...,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.333333
1,4wZwXhoxRIA,technology,Bro “how to talk to woman in 6 steps” is so re...,0,2025-09-10 23:44:24+00:00,bro “how talk woman 6 steps” relatable,neutral,0,12,5,...,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.000000
2,4wZwXhoxRIA,technology,I was gonna say does it give you the drinks fo...,0,2025-09-09 16:25:03+00:00,wa gonna say doe give drink free?😭,positive,1,12,7,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,4wZwXhoxRIA,technology,Anyone gonna talk abt what was o. His pc,0,2025-09-08 22:33:58+00:00,anyone gonna talk abt wa o. pc,neutral,0,9,3,...,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.000000
4,4wZwXhoxRIA,technology,HOW IS EVERYONE NOT TALKING ABOUT HIS SEARCH?!...,0,2025-09-08 12:17:37+00:00,"everyone not talking search?!""how tell ur girl...",positive,1,15,6,...,0.0,0.0,0.0,0.000000,0.0,0.500000,0.0,0.0,0.0,0.500000


In [12]:
df['text_clean'] = df['text_clean'].fillna(df['text'])


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
import scipy.sparse as sp
import numpy as np

# -----------------------------
# 1️⃣ Numeric features
# -----------------------------
numeric_cols = [col for col in df.columns if col not in [
    'video_id', 'category', 'text', 'text_clean', 'sentiment', 
    'dominant_emotion', 'published_at', 'sentiment_numeric'
]]

X_numeric = df[numeric_cols]
y = df['sentiment_numeric']

# Scale numeric features
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# -----------------------------
# 2️⃣ Train-test split (reuse same split)
# -----------------------------
X_train_num, X_test_num, y_train, y_test, train_idx, test_idx = train_test_split(
    X_numeric_scaled, y, df.index, test_size=0.2, random_state=42, stratify=y
)

# -----------------------------
# 3️⃣ Text features
# -----------------------------
df_train_text = df.loc[train_idx, 'text_clean']
df_test_text = df.loc[test_idx, 'text_clean']

# -----------------------------
# 4️⃣ Experiment 3: Tune max_features
# -----------------------------
best_accuracy = 0
best_run_info = {}

def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, n_estimators=100, max_depth=15):
    global best_accuracy, best_run_info

    # Vectorization
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    # Fit and transform text
    X_train_vec = vectorizer.fit_transform(df_train_text)
    X_test_vec = vectorizer.transform(df_test_text)

    # Combine with numeric features
    X_train_sparse = sp.hstack([X_train_vec, sp.csr_matrix(X_train_num)])
    X_test_sparse = sp.hstack([X_test_vec, sp.csr_matrix(X_test_num)])

    # -----------------------------
    # MLflow logging
    # -----------------------------
    with mlflow.start_run() as run:
        run_name = f"{vectorizer_type}_{ngram_range}_{vectorizer_max_features}feat"
        mlflow.set_tag("mlflow.runName", run_name)
        mlflow.set_tag("experiment_type", "max_features_tuning")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Log parameters
        mlflow.log_params({
            "vectorizer_type": vectorizer_type,
            "ngram_range": ngram_range,
            "vectorizer_max_features": vectorizer_max_features,
            "numeric_features_count": X_train_num.shape[1],
            "n_estimators": n_estimators,
            "max_depth": max_depth
        })

        # Train RandomForest
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_leaf=2,
            max_features='sqrt',
            random_state=42,
            class_weight='balanced',
            n_jobs=-1
        )
        model.fit(X_train_sparse, y_train)

        # Predictions & metrics
        y_pred = model.predict(X_test_sparse)
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8,6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_type}, {vectorizer_max_features} features")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_type}_{vectorizer_max_features}feat")

        # Update best run
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_run_info = {
                "vectorizer": vectorizer_type,
                "ngram_range": ngram_range,
                "max_features": vectorizer_max_features,
                "n_estimators": n_estimators,
                "max_depth": max_depth,
                "accuracy": accuracy,
                "run_id": run.info.run_id
            }


# -----------------------------
# 5️⃣ Run experiments for different max_features
# -----------------------------
max_features_list = [2000, 5000, 8000, 10000, None]  # None = all features
for max_feat in max_features_list:
    run_experiment(
        vectorizer_type="TF-IDF",
        ngram_range=(1,3),
        vectorizer_max_features=max_feat,
        n_estimators=100,
        max_depth=15
    )

print("✅ Best Run Info:")
print(best_run_info)


2025/09/29 16:30:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_2000feat at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2/runs/8131e51ce83d400cb2c50fe5728c0c29
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2


2025/09/29 16:31:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_5000feat at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2/runs/95e497ead06f40779c922400ddc746dc
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2


2025/09/29 16:32:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_8000feat at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2/runs/1567880ad18d45789d155166e53f9a97
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2


2025/09/29 16:33:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_10000feat at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2/runs/a7345d7e1b0b437d86e5c3c79a0accc3
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2


2025/09/29 16:35:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_Nonefeat at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2/runs/36483db5dbb049179b614684d078ccea
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/2
✅ Best Run Info:
{'vectorizer': 'TF-IDF', 'ngram_range': (1, 3), 'max_features': 2000, 'n_estimators': 100, 'max_depth': 15, 'accuracy': 0.6549914202079338, 'run_id': '8131e51ce83d400cb2c50fe5728c0c29'}


In [11]:
df[df['text_clean'].isna()]

,video_id,category,text,likes,published_at,text_clean,sentiment,sentiment_numeric,word_count,num_stop_words,...,fear,anger,anticipation,trust,surprise,positive,negative,sadness,disgust,joy
242,4wZwXhoxRIA,technology,This is a\nThis is a\nThis is a,0,2025-05-14 05:06:05+00:00,NaN,neutral,0,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1138,4wZwXhoxRIA,technology,This is a,0,2025-01-11 21:18:49+00:00,NaN,neutral,0,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8183,GJZAcj4P99g,music,Just now,0,2025-09-04 10:33:40+00:00,NaN,neutral,0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11888,sUf2PtEZris,music,Ll up,0,2025-08-29 00:48:21+00:00,NaN,neutral,0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12066,sUf2PtEZris,music,I am,0,2025-08-27 12:21:28+00:00,NaN,neutral,0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12615,sUf2PtEZris,music,It's me,1,2025-08-23 13:07:47+00:00,NaN,neutral,0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15230,sUf2PtEZris,music,Ll a,0,2025-08-08 17:48:11+00:00,NaN,neutral,0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17420,sUf2PtEZris,music,ONCE AGAIN,0,2025-07-29 17:31:42+00:00,NaN,neutral,0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17774,sUf2PtEZris,music,So what is,1,2025-07-28 09:02:07+00:00,NaN,neutral,0,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18276,sUf2PtEZris,music,I m too,0,2025-07-26 12:09:41+00:00,NaN,neutral,0,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
